In [1]:
#import python packages
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import pickle

%matplotlib inline

In [2]:
#import EKG analysis packages
import wave
import challengeOld

ImportError: No module named 'detect_peaks'

In [123]:
records = wave.getRecords('All')
feat_list = []
for record in records[0]:
    signal = challengeOld.Signal(record, wave.load(record))
    feat_list.append(challengeOld.feature_extract(signal))


In [98]:
variance = data_test[:,3]
mean_variance = np.mean(variance)
normalize = (variance - mean_variance)/mean_variance
data_test[:,3] = normalize

variance = data_train[:,3]
mean_variance = np.mean(variance)
normalize = (variance - mean_variance)/mean_variance
data_train[:,3] = normalize

In [99]:
#change the answer to integer classes
answer_test_int = []
answer_train_int = []
for answer in answer_test:
    if answer == 'N':
        answer_test_int.append(0)
    elif answer == 'O':
        answer_test_int.append(1)
    elif answer == 'A':
        answer_test_int.append(2)
    elif answer == '~':
        answer_test_int.append(3)

for answer in answer_train:
    if answer == 'N':
        answer_train_int.append(0)
    elif answer == 'O':
        answer_train_int.append(1)
    elif answer == 'A':
        answer_train_int.append(2)
    elif answer == '~':
        answer_train_int.append(3)


In [109]:
len(data_test[0])

4

In [110]:
EPOCHS = 100
BATCH_SIZE = 128
learning_rate = 0.0001
num_examples = len(data_train)

In [111]:
X = tf.placeholder(tf.float32)
y = tf.placeholder(tf.int32)
keep_prob = tf.placeholder(tf.float32)
one_hot_y = tf.one_hot(y, 4)

In [112]:
fw1 = tf.Variable(tf.truncated_normal([4, 8], mean=0, stddev=0.1))
fb1 = tf.Variable(tf.zeros([8]))
fw2 = tf.Variable(tf.truncated_normal([8, 8], mean=0, stddev=0.1))
fb2 = tf.Variable(tf.zeros([8]))
fw3 = tf.Variable(tf.truncated_normal([8, 4], mean=0, stddev=0.1))
fb3 = tf.Variable(tf.zeros([4]))


In [113]:
#fully connect layer 1, 4 => 8
fully_layer_1 = tf.add(tf.matmul(X, fw1), fb1)
fully_layer_1 = tf.nn.relu(fully_layer_1)
#fully_layer_1 = tf.nn.dropout(fully_layer_1, keep_prob)

#fully connect layer 2, 8 => 8
fully_layer_2 = tf.add(tf.matmul(fully_layer_1, fw2), fb2)
fully_layer_2 = tf.nn.relu(fully_layer_2)
#fully_layer_2 = tf.nn.dropout(fully_layer_2, keep_prob)

#output layer => 4 => 4
logits = tf.add(tf.matmul(fully_layer_2, fw3), fb3)


In [114]:
#train the model
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss_operation)

#accuracy and loss
correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(one_hot_y,1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

In [115]:
def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_X, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={X: batch_X, y: batch_y})
        total_accuracy += (accuracy * len(batch_X))
    return total_accuracy / num_examples

In [116]:
from sklearn.utils import shuffle
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    data_train, answer_train_int = shuffle(data_train, answer_train_int)
    for i in range(EPOCHS):
        sess.run(training_op, feed_dict={X: data_train, y: answer_train_int})
        
        validation_accuracy = evaluate(data_test, answer_test_int)
        print ('EPOCHS {}..'.format(i+1))
        print ('validation_accuracy = {:.3f}'.format(validation_accuracy))
        print ()
        
    saver.save(sess, 'model')
    print ('model_saved')

EPOCHS 1..
validation_accuracy = 0.271

EPOCHS 2..
validation_accuracy = 0.277

EPOCHS 3..
validation_accuracy = 0.286

EPOCHS 4..
validation_accuracy = 0.290

EPOCHS 5..
validation_accuracy = 0.293

EPOCHS 6..
validation_accuracy = 0.302

EPOCHS 7..
validation_accuracy = 0.302

EPOCHS 8..
validation_accuracy = 0.304

EPOCHS 9..
validation_accuracy = 0.313

EPOCHS 10..
validation_accuracy = 0.314

EPOCHS 11..
validation_accuracy = 0.317

EPOCHS 12..
validation_accuracy = 0.321

EPOCHS 13..
validation_accuracy = 0.322

EPOCHS 14..
validation_accuracy = 0.328

EPOCHS 15..
validation_accuracy = 0.333

EPOCHS 16..
validation_accuracy = 0.337

EPOCHS 17..
validation_accuracy = 0.341

EPOCHS 18..
validation_accuracy = 0.342

EPOCHS 19..
validation_accuracy = 0.345

EPOCHS 20..
validation_accuracy = 0.350

EPOCHS 21..
validation_accuracy = 0.355

EPOCHS 22..
validation_accuracy = 0.357

EPOCHS 23..
validation_accuracy = 0.358

EPOCHS 24..
validation_accuracy = 0.364

EPOCHS 25..
validation_ac

In [124]:
with tf.Session() as sess:
    
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    predictions = tf.argmax(logits,1)
    print(sess.run(predictions, feed_dict={X: data_test}))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 3 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0
 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 3 0 0 0 0 0 0 